Ref: https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3

In [2]:
!pip install nltk


In [3]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [43]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import math

In [24]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:]] = freq_table

    return frequency_matrix

In [25]:
para= "My thanks, also, to some of the speakers on this platform who, referring to the delegates from the Orient, have told you that these men from far-off nations may well claim the honor of bearing to different lands the idea of toleration. I am proud to belong to a religion which has taught the world both tolerance and universal acceptance. We believe not only in universal toleration, but we accept all religions as true. I am proud to belong to a nation which has sheltered the persecuted and the refugees of all religions and all nations of the earth. I am proud to tell you that we have gathered in our bosom the purest remnant of the Israelites, who came to Southern India and took refuge with us in the very year in which their holy temple was shat­tered to pieces by Roman tyranny. I am proud to belong to the religion which has sheltered and is still fostering the remnant of the grand Zoroastrian nation. I will quote to you, brethren, a few lines from a hymn which I remember to have repeated from my earliest boyhood, which is every day repeated by millions of human beings: “As the different streams having their sources in different paths which men take through different tendencies, various though they appear, crooked or straight, all lead to Thee.”"

In [70]:
len(para)

1262

In [26]:
sentences = sent_tokenize(para)
total_documents = len(sentences)
freq_matrix=_create_frequency_matrix(sentences)

In [37]:
#freq_matrix

In [33]:
#Create TFIDF matrix
def tf_matrix(freq_matrix):
    tfMatrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tfMatrix[sent] = tf_table
    #print(tf_table)
    return tfMatrix

In [34]:
tfMatrix=tf_matrix(freq_matrix)
print(tfMatrix)

{'My thanks, also, to some of the speakers on this platform who, referring to the delegates from the Orient, have told you that these men from far-off nations may well claim the honor of bearing to different lands the idea of toleration.': {'thank': 0.043478260869565216, ',': 0.17391304347826086, 'also': 0.043478260869565216, 'speaker': 0.043478260869565216, 'thi': 0.043478260869565216, 'platform': 0.043478260869565216, 'refer': 0.043478260869565216, 'deleg': 0.043478260869565216, 'orient': 0.043478260869565216, 'told': 0.043478260869565216, 'men': 0.043478260869565216, 'far-off': 0.043478260869565216, 'nation': 0.043478260869565216, 'may': 0.043478260869565216, 'well': 0.043478260869565216, 'claim': 0.043478260869565216, 'honor': 0.043478260869565216, 'bear': 0.043478260869565216, 'differ': 0.043478260869565216, 'land': 0.043478260869565216, 'idea': 0.043478260869565216, 'toler': 0.043478260869565216, '.': 0.043478260869565216}, 'I am proud to belong to a religion which has taught the

In [35]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [40]:
count_doc_per_words=_create_documents_per_words(freq_matrix)

In [44]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [45]:
idfMatrix= _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)

In [46]:
print(idfMatrix)

{'My thanks, also, to some of the speakers on this platform who, referring to the delegates from the Orient, have told you that these men from far-off nations may well claim the honor of bearing to different lands the idea of toleration.': {'thank': 0.8450980400142568, ',': 0.24303804868629442, 'also': 0.8450980400142568, 'speaker': 0.8450980400142568, 'thi': 0.8450980400142568, 'platform': 0.8450980400142568, 'refer': 0.8450980400142568, 'deleg': 0.8450980400142568, 'orient': 0.8450980400142568, 'told': 0.8450980400142568, 'men': 0.5440680443502757, 'far-off': 0.8450980400142568, 'nation': 0.36797678529459443, 'may': 0.8450980400142568, 'well': 0.8450980400142568, 'claim': 0.8450980400142568, 'honor': 0.8450980400142568, 'bear': 0.8450980400142568, 'differ': 0.5440680443502757, 'land': 0.8450980400142568, 'idea': 0.8450980400142568, 'toler': 0.36797678529459443, '.': 0.0}, 'I am proud to belong to a religion which has taught the world both tolerance and universal acceptance.': {'proud

In [47]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [49]:
tfidfMatrix= _create_tf_idf_matrix(tfMatrix, idfMatrix)

In [50]:
print(tfidfMatrix)

{'My thanks, also, to some of the speakers on this platform who, referring to the delegates from the Orient, have told you that these men from far-off nations may well claim the honor of bearing to different lands the idea of toleration.': {'thank': 0.03674339304409812, ',': 0.0422674867280512, 'also': 0.03674339304409812, 'speaker': 0.03674339304409812, 'thi': 0.03674339304409812, 'platform': 0.03674339304409812, 'refer': 0.03674339304409812, 'deleg': 0.03674339304409812, 'orient': 0.03674339304409812, 'told': 0.03674339304409812, 'men': 0.023655132363055463, 'far-off': 0.03674339304409812, 'nation': 0.015998990664982365, 'may': 0.03674339304409812, 'well': 0.03674339304409812, 'claim': 0.03674339304409812, 'honor': 0.03674339304409812, 'bear': 0.03674339304409812, 'differ': 0.023655132363055463, 'land': 0.03674339304409812, 'idea': 0.03674339304409812, 'toler': 0.015998990664982365, '.': 0.0}, 'I am proud to belong to a religion which has taught the world both tolerance and universal

In [52]:
def _score_sentences(tfidfMatrix):
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tfidfMatrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [53]:
score= _score_sentences(tfidfMatrix)

In [54]:
print(score)

{'My thanks, also, to some of the speakers on this platform who, referring to the delegates from the Orient, have told you that these men from far-off nations may well claim the honor of bearing to different lands the idea of toleration.': 0.03244406150146935, 'I am proud to belong to a religion which has taught the world both tolerance and universal acceptance.': 0.04368338621985437, 'We believe not only in universal toleration, but we accept all religions as true.': 0.05527756902975931, 'I am proud to belong to a nation which has sheltered the persecuted and the refugees of all religions and all nations of the earth.': 0.05037345402944012, 'I am proud to tell you that we have gathered in our bosom the purest remnant of the Israelites, who came to Southern India and took refuge with us in the very year in which their holy temple was shat\xadtered to pieces by Roman tyranny.': 0.031132126635430565, 'I am proud to belong to the religion which has sheltered and is still fostering the rem

In [56]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [57]:
averageScore= _find_average_score(score)

In [58]:
print(averageScore)

0.040484113900856565


In [66]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:] in sentenceValue and sentenceValue[sentence[:]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [76]:
threshold=1.2*averageScore
summary= _generate_summary(sentences, score, threshold)

In [77]:
print(summary)

 We believe not only in universal toleration, but we accept all religions as true. I am proud to belong to a nation which has sheltered the persecuted and the refugees of all religions and all nations of the earth.


In [78]:
len(summary)

214